In [ ]:
import pandas as pd, re, unicodedata, difflib

def normalize(text):
    if pd.isna(text): return ''
    s = unicodedata.normalize('NFKD', str(text)).lower()
    s = re.sub(r'\b(ооо|пао|ао|зао|pjsc|ojsc|ltd|gmbh|гк)\b', ' ', s)
    s = re.sub(r'[«»"“”\'‘’]', '', s)
    s = re.sub(r'[-–]', ' ', s)
    s = re.sub(r'[^\w\s]', '', s)
    return re.sub(r'\s+', ' ', s).strip()

reviews = pd.read_csv('reviews_with_sentiment.csv')
info    = pd.read_csv('cian_zastroischiki_info_all.csv')

reviews['dev_norm'] = reviews['developer'].apply(normalize)
info['dev_norm']    = info['developer_name'].apply(normalize)

manual_map_raw = {
    'гк главстрой': 'Главстрой Санкт-Петербург',
    'главстрои':   'Главстрой Санкт-Петербург',
    'группа пик':  'пик',
    'пик группа':  'пик',
    'группа фск':  'фск'
}

manual_map = {normalize(k): normalize(v) for k, v in manual_map_raw.items()}

reviews['dev_norm'] = reviews['dev_norm'].replace(manual_map)

# добавляем строки для 7 застройщиков вручную так как у них катсомные страницы на циан/нет необходимой информации
pik_row = {
    'dev_norm':           normalize('пик'),
    'year_of_foundation': 1994,
    'cnt_houses_built':   1268,
    'estate_class_list':  'Эконом; Комфорт; Бизнес',
    'avg_sqm_price':      329_270
}

fsk_row = {
    'dev_norm':           normalize('фск'),
    'year_of_foundation': 2005,
    'cnt_houses_built':   202,
    'estate_class_list':  'Комфорт; Бизнес',
    'avg_sqm_price':      341_363
}

polis_row = {
    'dev_norm':           normalize('полис'),
    'year_of_foundation': 2010,
    'cnt_houses_built':   68,
    'estate_class_list':  'Комфорт',
    'avg_sqm_price':      150_500
}

october_row = {
    'dev_norm':           normalize('october group'),
    'year_of_foundation': 2022,
    'cnt_houses_built':   0,
    'estate_class_list':  'Бизнес',
    'avg_sqm_price':      637_960
}

vozrozhdenie_row = {
    'dev_norm':           normalize('возрождение северо запад иск'),
    'year_of_foundation': 2000,
    'cnt_houses_built':   1,
    'estate_class_list':  'Комфорт',
    'avg_sqm_price':      190_000
}

interrost_row = {
    'dev_norm':           normalize('интеррост'),
    'year_of_foundation': 2014,
    'cnt_houses_built':   2,
    'estate_class_list':  'Комфорт',
    'avg_sqm_price':      278_818
}

optima_row = {
    'dev_norm':           normalize('optima development'),
    'year_of_foundation': 2007,
    'cnt_houses_built':   5,
    'estate_class_list':  'Премиум',
    'avg_sqm_price':      632_677
}

forma_row = {
    'dev_norm':           normalize('forma'),
    'year_of_foundation': 2021,
    'cnt_houses_built':   5,
    'estate_class_list':  'Премимум',
    'avg_sqm_price':      1_033_485
}

info = pd.concat([info, pd.DataFrame([pik_row, fsk_row, polis_row, october_row, vozrozhdenie_row, interrost_row, optima_row, forma_row])],
                 ignore_index=True)


cols_keep = ['dev_norm', 'year_of_foundation', 'cnt_houses_built',
             'estate_class_list', 'avg_sqm_price']

info_one = (info[cols_keep]
            .sort_values('year_of_foundation', ascending=False, na_position='last')
            .groupby('dev_norm', as_index=False)
            .agg('first'))


joined = (reviews
          .merge(info_one, on='dev_norm', how='left'))

# Проверяем
print('Строк до/после:', len(reviews), len(joined))
print('Доля без цены м²:', joined['avg_sqm_price'].isna().mean().round(3))


In [ ]:
joined.head(5)

In [ ]:
df = joined

In [ ]:
"""
One-hot-encoding для столбца `estate_class_list`
• итоговые колонки строго:  'Эконом', 'Комфорт', 'Бизнес', 'Премиум'
• если класс не указан → 0
• результат →  joined_with_onehot.csv
"""


canon = {
    "эконом"  : "Эконом",
    "комфорт" : "Комфорт",
    "бизнес"  : "Бизнес",
    "премиум" : "Премиум",          # сюда же можно отнести "элитный"
    "элит"    : "Премиум"           # на случай «Элит» / «Элитный»
}

# заготовим все 4 столбца нулями
for col in canon.values():
    df[col] = 0


def token_set(text: str) -> set[str]:
    """
    возвращает множество "сырых" токенов в нижнем регистре
      эконом-класс, Комфорт / бизнес → {'эконом', 'комфорт', 'бизнес'}
    """
    if pd.isna(text):
        return set()
    txt = unicodedata.normalize("NFKD", str(text)).lower()
    txt = re.sub(r"[\\/]", ";", txt)        # "/" "\" → ;
    txt = re.sub(r"[,\s]+", ";", txt)       # запятые и лишние пробелы → ;
    parts = [p.strip() for p in txt.split(";") if p.strip()]
    tokens = set()
    for p in parts:
        # обрежем возможное "-класс", "-класс", " класс"
        p = re.sub(r"\s*-?-?класс$", "", p)
        tokens.add(p)
    return tokens

for idx, raw in enumerate(df["estate_class_list"]):
    toks = token_set(raw)
    # делаем 1 там, где токен распознан → колонки canon[token]
    for t in toks:
        if t in canon:
            df.at[idx, canon[t]] = 1

#добавляем поле с кол-вом лет на рынке
CURRENT_YEAR = 2025
df["years_on_market"] = CURRENT_YEAR - df["year_of_foundation"]
df.to_csv("joined_with_onehot_to_use_final.csv", index=False)